# Guardian Angel - Kids and Caregivers Recognition

+ Objective
    - Primary Goal: To reliably identify kids and caregivers in a video stream, enabling further analysis in subsequent models and ensuring the system can discern if a room is occupied by a caregiver or if children are unattended.

+ Requirements: 
    - The model must be lightweight to ensure continuous operation without significant resource consumption. Stability across various conditions is paramount to avoid downtime or errors in identification.

+ Plan A: Face Recognition

+ Objective: Implement a face recognition system to identify and name each child from video frame inputs, detailing their face locations within the frame.

+ Technologies and Methods:

    1. Convolutional Neural Networks (CNNs): 
        - Utilize CNNs for robust feature extraction from facial data in video frames.

    2. DeepFace: 
        - Leverage the DeepFace framework for simplified face recognition process, supporting various underlying models like VGG-Face, Google FaceNet, OpenFace, Facebook DeepFace, and more.

    3. Face Recognition Package: 
        - Implement this popular Python library, which provides practical and efficient face recognition capabilities.

    4. Siamese Neural Networks: 
        - Apply Siamese networks for learning unique features from images, aiding in the effective differentiation between individuals.

    5. Few Shot Learning: 
        - Integrate few-shot learning techniques to effectively perform face recognition with limited data samples.

+ Plan B: Body Recognition

+ Objective: Develop a body recognition system to identify and name each child by analyzing their entire body within video frames, pinpointing the location of each identified individual.

+ Technologies and Methods:

    1. Pose Estimation Models: 
        - Utilize models that perform pose estimation to help in identifying body shapes and postures from video frames.

    2. Segmentation Algorithms: 
        - Apply segmentation techniques to separate and identify individual children based on their body outlines.

    3. Convolutional Neural Networks (CNNs): 
        - Use CNNs for extracting detailed features from the entire body, enhancing the accuracy of body recognition.

    4. Feature Matching Techniques: 
        - Employ feature matching to compare and identify bodies based on unique physical characteristics.

    5. Ensemble Learning: 
        - Combine multiple models and techniques to improve the robustness and accuracy of the body recognition system.

+ MVP Development Strategy

1. Data Collection and Preparation:
    + Task: Select 60 frames from a 12-minute recorded video, ensuring these frames represent a diverse range of scenarios involving kids and caregivers in different areas of the room, engaged in various activities.
    
    + Purpose: To create a balanced and representative dataset for initial training, focusing on variability to enhance model robustness.

2. Model Selection and Training: e.g. (Yolov8s + Byte Track)
    + Task: Choose a lightweight yet effective pre-trained computer vision model suited for object detection tasks.

    + Purpose: By fine-tuning a pre-trained model on your specific dataset, you can leverage learned features while minimizing the computational load, aligning with the goal of maintaining a light and stable system.

3. Model Testing and Validation:
    + Task: Apply the trained model to the entire 12-minute video to evaluate its performance in real-world conditions, focusing on the accuracy of identifying kids and caregivers and the model's stability over continuous operation.

    + Purpose: This step is crucial for assessing the model's readiness for real-world application and identifying any adjustments needed to improve accuracy or reduce resource consumption.

my supervior said we not have many images for training but we have same videos for training but we make testing we only will have one image for kid and caregiver so how handel this problem?

Step 1: Download and Prepare the Dataset

In [2]:
# !pip install roboflow
from roboflow import Roboflow

In [2]:
def load_roboflow_dataset(api_key, workspace, project, version, model_format="yolov9"):
    """
    Load a dataset from Roboflow.

    Parameters:
    - api_key (str): Your Roboflow API key.
    - workspace (str): The workspace name.
    - project (str): The project name.
    - version (str): The dataset version.
    - model_format (str): The format to download the dataset in (default is 'yolov5').

    Returns:
    - dataset_path (str): The local path to the downloaded dataset.
    """
    # Initialize Roboflow
    rf = Roboflow(api_key=api_key)
    
    # Get the project
    project = rf.workspace(workspace).project(project)
    
    # Download the dataset
    dataset = project.version(version).download(model_format)
    
    # Return the local path to the dataset
    return dataset.location

# Define your Roboflow API details
ROBOFLOW_API_KEY = 'XD1WXWBov1OCGhi0KENH'
ROBOFLOW_WORKSPACE = 'guardian-angel'
ROBOFLOW_PROJECT = 'nursery_project'
ROBOFLOW_VERSION = '1'
model_format = 'yolov9'

# Load the dataset
dataset_path = load_roboflow_dataset(ROBOFLOW_API_KEY, ROBOFLOW_WORKSPACE, ROBOFLOW_PROJECT, ROBOFLOW_VERSION, model_format)

print(f"Dataset downloaded to: {dataset_path}")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to nursery_project-1 in yolov9:: 100%|██████████| 153/153 [00:00<00:00, 261.29it/s]


Dataset downloaded to: d:\Projects\Guardian Angel\Kids and Caregivers Recognition\nursery_project-1


path of dataset

+ yolov9\nursery_project-1
    - train
        - images
        - labels
    - valid
        - images
        - labels
    - data.yaml
    - README.dataset
    - README.roboflow

Step 2: Train YOLOv8 for Object Detection

Train YOLOv8 for Object Detection: Detect kids and caregivers in video frames.

In [7]:
from ultralytics import YOLO
import os

In [8]:
model = YOLO('yolov8s.pt')
model

100%|██████████| 21.5M/21.5M [00:06<00:00, 3.38MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [9]:
home = os.getcwd()
home

'd:\\Projects\\guardian_angel\\kids_and_Caregivers_recognition'

In [10]:
training_results = model.train(data=rf"{home}/nursery_project-1/data.yaml", 
                               epochs=20, 
                               imgsz=640, 
                               batch=8, 
                               lr0=0.01, 
                               dropout=0.4, 
                               project=rf"{home}/training_results")
training_results

New https://pypi.org/project/ultralytics/8.2.16 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.15  Python-3.11.4 torch-2.0.1+cpu CPU (Intel Core(TM) i7-8665U 1.90GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=d:\Projects\guardian_angel\kids_and_Caregivers_recognition/nursery_project-1/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=d:\Projects\guardian_angel\kids_and_Caregivers_recognition/training_results, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, st

COMET WARNING: Can not parse empty Comet API key


WARNING  Comet installed but not initialized correctly, not logging this run. Comet.ml requires an API key. Please provide as the first argument to Experiment(api_key) or as an environment variable named COMET_API_KEY 
TensorBoard: Start with 'tensorboard --logdir d:\Projects\guardian_angel\kids_and_Caregivers_recognition\training_results\train5', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning D:\Projects\guardian_angel\kids_and_Caregivers_recognition\nursery_project-1\train\labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]Scanning D:\Projects\guardian_angel\kids_and_Caregivers_recognition\nursery_project-1\train\labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
Got processor for bboxes, but no transform to process it.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      20/20         0G     0.9107     0.5911      1.102         17        640: 100%|██████████| 8/8 [01:31<00:00, 11.50s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:07<00:00,  7.88s/it]


                   all         14        107      0.974      0.974       0.99      0.689

20 epochs completed in 0.659 hours.
Optimizer stripped from d:\Projects\guardian_angel\kids_and_Caregivers_recognition\training_results\train5\weights\last.pt, 22.5MB
Optimizer stripped from d:\Projects\guardian_angel\kids_and_Caregivers_recognition\training_results\train5\weights\best.pt, 22.5MB

Validating d:\Projects\guardian_angel\kids_and_Caregivers_recognition\training_results\train5\weights\best.pt...
Ultralytics YOLOv8.2.15  Python-3.11.4 torch-2.0.1+cpu CPU (Intel Core(TM) i7-8665U 1.90GHz)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:06<00:00,  6.74s/it]


                   all         14        107      0.974      0.974       0.99      0.689
             caregiver         14         11          1       0.99      0.995      0.684
                  kids         14         96      0.947      0.958      0.985      0.694
Speed: 3.0ms preprocess, 462.3ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to d:\Projects\guardian_angel\kids_and_Caregivers_recognition\training_results\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002567EB3B790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

### Step 3: Train EfficientNet for Classification

Use EfficientNet to classify detected faces into predefined classes (folders A, B, C, D, and Nanny representing each kid's class).

Train EfficientNet for Classification: Classify detected faces into classes A, B, C, D, and Nanny.

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'dataset_classification',  # path to your dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'dataset_classification',  # path to your dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Build EfficientNet model
EfficientNetB0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
efficientnet_model = Sequential([
    EfficientNetB0,
    GlobalAveragePooling2D(),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
efficientnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
efficientnet_model.fit(train_generator, epochs=10, validation_data=validation_generator)

loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Save the trained model
efficientnet_model.save('efficientnet_model.h5')

### Step 4: Combine YOLOv8 and EfficientNet for Recognition

Integrate Detection and Classification: Use YOLOv8 for detection. If a kid is detected, classify them using EfficientNet. If a caregiver is detected, assign them to the "Nanny" class.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array


In [ ]:
# Load YOLOv8 model
yolo_model = YOLO('yolov8s.pt')  # Load trained YOLOv8 model

# Load EfficientNet model
efficientnet_model = load_model('efficientnet_model.h5')

# Define class labels corresponding to folders A, B, C, D, and Nanny
class_labels = ['Class A', 'Class B', 'Class C', 'Class D', 'Nanny']

def detect_and_classify(video_path, yolo_model, efficientnet_model, class_labels):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLOv8 for detection
        results = yolo_model.predict(frame)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy)
                class_id = int(box.cls)
                confidence = box.conf.item()
                
                if class_id == 1:  # Assuming class 1 is the kid
                    # Extract face for classification
                    face = frame[y1:y2, x1:x2]
                    face_resized = cv2.resize(face, (224, 224))
                    face_array = img_to_array(face_resized) / 255.0
                    face_array = np.expand_dims(face_array, axis=0)
                    
                    # Classify the face using EfficientNet
                    predictions = efficientnet_model.predict(face_array)
                    class_index = np.argmax(predictions)
                    class_name = class_labels[class_index]
                else:
                    class_name = 'Nanny'
                
                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{class_name} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        cv2.imshow('Detection and Classification', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Path to the video file
video_path = 'path/to/video.mp4'

# Perform detection and classification on the video
detect_and_classify(video_path, yolo_model, efficientnet_model, class_labels)